<a href="https://colab.research.google.com/github/YaokunLin/ActuaryProgrammer/blob/main/naive_chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
data = {"intents":
 [
      {"tag": "greetings",
      "patterns": ["hello", "hey", "hi", "good day", "Greetings", "what's up?", "how is it going?"],
      "responses": ["Hello!", "Hey!", "What can I do for you?"],
     },

      {"tag": "goodbye",
       "patterns": ["cya", "See you later", "Goodbye", "I am Leaving", "Have a Good day", "bye", "cao", "see ya"],
       "responses": ["Sad to see you go :(", "Talk to you later", "Goodbye!"],
      },

      {"tag": "age",
        "patterns": ["how old", "how old is florian", "what is your age", "how old are you", "age?"],
        "responses": ["My owner Florian is 21 years old!", "21 years!"],
       },

      {"tag": "name",
         "patterns": ["what is your name", "what should I call you", "whats your name?", "who are you?", "Can you tell me your name?"],
         "responses": ["You can call me Neural!.", "I'm Neural!", "I'm Neural the assistant of Florian!"],
        },

      {"tag": "hobby",
          "patterns": ["what is your hobby", "what do you like to do in your free time?"],
          "responses": ["I like swimming"]
      },
          ]
}

In [ ]:
hftrhrthrjjytyjtyj

In [ ]:
sum([1,2])

3

In [ ]:
fhfhtrhrthsscsccscsdfvdcddfsdvsvdvdvdvsdv

True

In [ ]:
import pickle
import numpy as np
import nltk
from nltk.stem import WordNetLemmatizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD
import random

nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [ ]:
intents = data
lemmatizer = WordNetLemmatizer()

words = []
classes = []
documents = []
ignore_letters = ['?', '!', '.', ',']

for intent in intents['intents']:
  for pattern in intent['patterns']:
    word_list = nltk.word_tokenize(pattern)
    words.extend(word_list)
    documents.append((word_list, intent[ 'tag']))
    if intent['tag'] not in classes:
      classes.append(intent['tag'])

words = [lemmatizer.lemmatize(word) for word in words if word not in ignore_letters]
words = sorted(set(words))
classes = sorted(set(classes))

In [ ]:
pickle.dump(words, open('words.pk1', 'wb'))
pickle.dump(classes, open('classes.pkl', 'wb'))

training = []
output_empty = [0] * len(classes)
for document in documents:
  bag = []
  word_patterns = document[0]
  word_patterns = [lemmatizer.lemmatize(word.lower()) for word in word_patterns]

  for word in words: bag.append(1) if word in word_patterns else bag.append(0)

  output_row = list(output_empty)
  output_row[classes.index(document[1])] = 1
  training.append([bag, output_row])

In [ ]:
random.shuffle(training)
training = np.array(training)


train_x = list(training[:, 0])
train_y = list(training[:, 1])
model = Sequential()

model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss="categorical_crossentropy", optimizer=sgd, metrics=['accuracy'])
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.model', hist)

print("Done")


<ipython-input-5-394cee6d3829>:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training = np.array(training)
/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/gradient_descent.py:108: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


Epoch 1/200
6/6 [==============================] - 1s 2ms/step - loss: 1.6242 - accuracy: 0.2593
Epoch 2/200
6/6 [==============================] - 0s 2ms/step - loss: 1.5222 - accuracy: 0.3704
Epoch 3/200
6/6 [==============================] - 0s 2ms/step - loss: 1.5603 - accuracy: 0.1852
Epoch 4/200
6/6 [==============================] - 0s 2ms/step - loss: 1.3834 - accuracy: 0.4444
Epoch 5/200
6/6 [==============================] - 0s 2ms/step - loss: 1.4191 - accuracy: 0.4444
Epoch 6/200
6/6 [==============================] - 0s 2ms/step - loss: 1.4231 - accuracy: 0.4444
Epoch 7/200
6/6 [==============================] - 0s 2ms/step - loss: 1.3264 - accuracy: 0.5185
Epoch 8/200
6/6 [==============================] - 0s 2ms/step - loss: 1.1758 - accuracy: 0.5185
Epoch 9/200
6/6 [==============================] - 0s 2ms/step - loss: 1.0421 - accuracy: 0.6667
Epoch 10/200
6/6 [==============================] - 0s 3ms/step - loss: 1.0318 - accuracy: 0.7778
Epoch 11/200
6/6 [===========

In [ ]:
def clean_up_sentence(sentence):
  sentence_words = nltk.word_tokenize(sentence)
  sentence_words = [lemmatizer.lemmatize(word) for word in sentence_words]
  return sentence_words

def bag_of_words(sentence):
  sentence_words = clean_up_sentence(sentence)
  bag = [0] * len(words)
  for w in sentence_words:
    for i, word in enumerate(words):
      if word == w: bag[i] = 1
  return np.array(bag)

def get_response(intents_list, intents):
   tag = intents_list[0]["intent"]
   list_of_intents = intents[ 'intents']

   for i in list_of_intents:
     if i['tag'] == tag:
       result = random.choice(i['responses'])
       break
   return result

ERROR_THRESHOLD = 0.25
def predict_class(sentence):
   bow = bag_of_words(sentence)
   res = model.predict(np.array([bow]))[0]
   results = [[i, r] for i, r in enumerate(res) if r > ERROR_THRESHOLD]

   results.sort(key=lambda x: x[1], reverse=True)
   return_list = []
   for r in results: return_list.append({ 'intent': classes[r[0]], 'probability': str(r[1])})
   return return_list



In [ ]:
print ("GO! Bot is runnin !")
while True:
  message = input("")
  ints = predict_class(message)
  res = get_response(ints, intents)
  print(res)

GO! Bot is runnin !
1/1 [==============================] - 0s 71ms/step
What can I do for you?
1/1 [==============================] - 0s 15ms/step
21 years!


In [ ]:
def outside_func1():
  a = []
  def inside_func1():
    a.append(1)

  inside_func1()
  print(a)
outside_func1()

[1]


In [ ]:
def outside_func2():
  b = 5
  def inside_func2():
    b = 6
  inside_func2()
  print(b)

outside_func2()

5


In [ ]:
a = ["", "   "]

"-".join(a).split("-")


['', '   ']

In [ ]:
not []

True

In [ ]:
def encode(strs):
        return ''.join('%d:' % len(s) + s for s in strs)

In [ ]:
encode(["aaaa", "bbb"])

'4:aaaa3:bbb'

In [ ]:
a=[1,2,3,1]
a.remove(1)
a

[2, 3, 1]

In [ ]:
"1".lower()

'1'

In [ ]:
min([1,2,3])

1

In [ ]:
min(float('inf'), 4)

4

In [ ]:
5 == 10

False

In [ ]:
a = [1, 2, 3, 4]

In [ ]:
a.pop()

4

In [ ]:
a

[1, 2, 3, 5]

In [ ]:
a.append(5)

In [ ]:
def isSimilar(s1, s2):
    if s1 == s2:
        return True

    s1, s2 = list(s1), list(s2)
    i = 0
    while i <= len(s1)-1:
        if s1[i] != s2[i]:
            break
        i += 1

    j = i + 1
    while j <= len(s1)-1:
        if s1[j] == s2[i]:
            break
        j += 1

    s1[i], s1[j] = s1[j], s1[i]
    return s1 == s2

In [ ]:
isSimilar("tars", "rats")

True

In [ ]:
isSimilar("rats", "arts")

True

In [ ]:
isSimilar("tars", "arts")

False

In [ ]:
isSimilar("star", "tars"), isSimilar("star", "rats"), isSimilar("star", "arts")

(False, False, False)

In [ ]:
a = "abc"

def change_it(x):
  x[0], x[-1] = x[-1], x[0]
  print(x, "from func")

change_it(a)
a

In [ ]:
edges = [[0,1],[1,2],[3,4]]
def union(x, y):
  return (str(x) + ", " + str(y))
list(map(union, edges))

In [ ]:
import numpy as np
np.append([[1, 2, 3], [4, 5, 6]], [[7, 8, 9]], axis=0)

array([[1, 2, 3],
       [4, 5, 6],
       [7, 8, 9]])

In [ ]:
a = 5
a /= 10
a

0.5

In [ ]:
import numpy as np
l_orig = np.float32([4, 2])

In [ ]:
Xorign = np.linspace(0, 10, 21)

Yorig = l_orig[0] * Xorign + l_orig[1]
Yorig

array([ 2.,  4.,  6.,  8., 10., 12., 14., 16., 18., 20., 22., 24., 26.,
       28., 30., 32., 34., 36., 38., 40., 42.])

In [ ]:
points = np.array([[5,0], [4,0]])

for i, p in enumerate(points):
  print(i, p)

0 [5 0]
1 [4 0]


In [ ]:
def get_x():
  return 5

a = b = get_x()
a = 6
b

5

In [ ]:
import numpy as np
from scipy import stats
a = np.array([1, 2, 3, 4, 5, 1, 1])
stats.mode(a)[0][0]

IndexError: ignored

In [ ]:
b = [
    [1, 3],
     [1, 4],
      [2, 3]]
stats.mode(b, axis=0)

ModeResult(mode=array([1, 3]), count=array([2, 2]))

In [ ]:
#https://www.1point3acres.com/bbs/thread-1036061-1-1.html
arr = [1, 2, 3, 4]
ans = 0

for i in range(len(arr)):
  for j in range(i+1, len(arr)):
    left = arr[i] ^ arr[j]
    right = arr[i] & arr[j]
    if left > right:
      ans += 1
      print(left, right)

print(ans)

3 0
2 1
5 0
6 0
7 0
5


In [ ]:
#https://www.1point3acres.com/bbs/thread-1036061-1-1.htmlvs
select cat.title as category, prod.title, prod.total_stock
from
categories as cat
join
products as prod
on
  prod.category_id = cat.id
where
  prod.total_stock > 10
order by
  cat.title asc,
  prod.title asc,
  prod.total_stock desc



#https://www.1point3acres.com/bbs/thread-1037053-1-1.html
SELECT categories.title, products.title, SUM(products.stock_number) as stock_number
FROM categories
INNER JOIN products
ON categories.id = products.category_id
GROUP BY categories.title, products.title
HAVING stock_number >10
ORDER BY categories.title ASC, products.title ASC

In [ ]:
#2964. Number of Divisible Triplet Sums
https://leetcode.com/problems/number-of-divisible-triplet-sums/description/

1

In [ ]:
# https://www.1point3acres.com/bbs/thread-1035167-1-1.html
import re
def getResponses(valid_auth_tokens, requests):
    responses = []
    for http_type, url in requests:
        params = re.findall(r"[\?&](\w+)=([^&]+)", url)
        token = [val for key, val in params if key == "token"]
        is_valid_token = token[0] in valid_auth_tokens if token else False
        response = "INVALID"
        if is_valid_token:
            if http_type == "POST":
                csrf = [val for key, val in params if key == "csrf"]
                is_valid_csrf = bool(csrf and re.match(r'^[a-z0-9]{8,}$', csrf[0]))
                if is_valid_csrf:
                    response = "VALID"
            else:
                # GET
                response = "VALID"
        if response == "VALID":
            response += "," + ",".join(f"{key},{value}" for key, value in params if key not in ("token", "csrf"))
        responses.append(response)
    return responses

valid_auth_tokens = ["ah37j2ha483u", "safh34yw0bp5", "ba34wyt8t902"]
requests = [
    ["GET", "https://example.com/?token=347sd6yk8iu2&name=alex"],
    ["GET", "https://example.com/?token=safh34yw0bp5&name=sam"],
    ["POST", "https://example.com/?token=safh34yw0bp5&name=alex"],
    ["POST", "https://example.com/?token=safh34yw0bp5&csrf=ak2sh32dy&name=chris"]
]

getResponses(valid_auth_tokens, requests)


['INVALID', 'VALID,name,sam', 'INVALID', 'VALID,name,chris']

In [ ]:
"qb".split("/")

['qb']

In [ ]:
"0" + "1"

'01'

In [ ]:
1+[0,1]

TypeError: unsupported operand type(s) for +: 'int' and 'list'

In [ ]:
def congratulations(super_awesome_devop):
  message = f"""     Congratulations, {super_awesome_devop} !!  🎉🎉🎉  """
  return message;
print(congratulations("Hope Wilson"))

     Congratulations, Hope Wilson !!  🎉🎉🎉  


In [ ]:
a = set([0, 1])
[element] = a

ValueError: too many values to unpack (expected 1)

In [ ]:
import numpy as np
def describe(x):
    print(
        "Count={count}\tMean={mean:.3f}\tstd={median:.3f}\n"
        "Min={min:.3f}\t25%={p25:.3f}\t50%={p50:.3f}\t75%={p75:<.3f}\tMax={max:.3f}".format(
            count=len(x),
            mean=np.mean(x),
            median=np.std(x),
            min=min(x),
            p25=np.percentile(x, 25),
            p50=np.median(x),
            p75=np.percentile(x, 75),
            max=max(x),
        )
    )

In [ ]:
describe(a)

Count=788	Mean=0.363	std=0.105
Min=0.263	25%=0.314	50%=0.334	75%=0.369	Max=1.526


In [ ]:
a = [0.5153268292429655,
 0.28722861813361156,
 0.33747883947602714,
 0.47752448581716317,
 0.3299523355681578,
 0.3071041568323393,
 0.2849190087038212,
 0.3939964345897855,
 0.28954182360286396,
 0.3266137821276008,
 0.3411882709422398,
 0.3020837761538984,
 0.3069446792281245,
 0.3539613141285828,
 0.3268913510296736,
 0.32266755520982243,
 0.29969312746322974,
 0.34383861598367543,
 0.29270035916734294,
 0.49684583191851767,
 0.2625493593901976,
 0.30315351210227875,
 0.3346656760489263,
 0.3866403314442356,
 0.35487250780857915,
 0.28929223661230524,
 0.32640423920650297,
 0.2840490294088365,
 0.314455996769382,
 0.37037264074449106,
 0.3333037014976812,
 0.29055426513062055,
 0.32605540784939513,
 0.36159426905701025,
 0.45332021068597667,
 0.30958696623880705,
 0.33458456239529183,
 0.39561767265991643,
 0.32414181198776987,
 0.327516359180139,
 0.34203178406978124,
 0.46315904252036294,
 0.29277881618969914,
 0.420985333154165,
 0.6216777634165248,
 0.3426268050397277,
 0.2913775493798115,
 0.3855651063637703,
 0.3347631023155461,
 0.3135360946346444,
 0.3975381991112185,
 0.37255480251105266,
 0.31105339429347173,
 0.31741028438684343,
 0.3215888016794375,
 0.31641442691547317,
 0.3562439301313699,
 0.30550538354914114,
 0.31893354412760133,
 0.3009862836506618,
 0.35408010589907896,
 0.2957159748466813,
 0.3169819171783286,
 0.48308351722985404,
 0.2933212291823816,
 0.30862046904676427,
 0.3718140775363026,
 0.5401050796460821,
 0.378148267796681,
 0.3712881396211961,
 0.2817088351391211,
 0.4838931555228299,
 0.36378456336543885,
 0.3601003958920234,
 0.3667577423118779,
 0.3748483619701571,
 0.3344739947817376,
 0.3299959524205929,
 0.31189959903061054,
 0.3132386514717237,
 0.3311410925871985,
 0.33481224188154785,
 0.338115664446957,
 0.3447261498525664,
 0.36745334938657837,
 0.3069501286889003,
 0.31734086777325815,
 0.28545574340242835,
 0.4190974520100415,
 0.3134945198336516,
 0.3551162626966506,
 0.32854575945998316,
 0.4118184715594543,
 0.3528502850964481,
 0.29726462669068204,
 0.34182284537916574,
 0.3172573521131801,
 0.31379169719851363,
 0.356059686577939,
 0.5156883863820126,
 0.30897432114257856,
 0.33549612869203466,
 0.5554364162671519,
 0.34677595103232156,
 0.35713270213570897,
 0.31532701214227743,
 0.3184733974380342,
 0.332163889684474,
 0.33973708089106874,
 0.3290857560405575,
 0.3558959709353425,
 0.3120807553220626,
 0.37971829723385314,
 0.3605546548620689,
 0.3144700290777867,
 0.28259775913162166,
 0.31442697359284416,
 0.33237234248925973,
 0.29812247543409226,
 0.3183341018110851,
 0.3826188132935631,
 0.3322692918779578,
 0.2885433229066511,
 0.30272677590693414,
 0.39963036027980187,
 0.3216903559024811,
 0.3959298516533177,
 0.3368129766142962,
 0.29982392930456986,
 0.36463203412145084,
 0.3158103261281353,
 0.3199253824855623,
 0.29686665507568294,
 0.35814404891225965,
 0.283307779016131,
 0.5741500421901478,
 0.3817137367015705,
 0.3311063927393965,
 0.3526496787713348,
 0.32892603049022334,
 0.37482766607348306,
 0.35454796177973835,
 0.34253167435894705,
 0.321852433984481,
 0.2979125399227277,
 0.3188595796709793,
 0.27143645569809666,
 0.3095843460839937,
 0.31051753602176624,
 0.33249106038167664,
 0.3286376801796997,
 0.42254407688700996,
 0.3320418555348289,
 0.35811252027695467,
 0.2829275980066044,
 0.28367901021459047,
 0.3129804231234465,
 0.33917128378276173,
 0.34742773439723906,
 0.32844950755976127,
 0.33716214765027064,
 0.3270864894641962,
 0.3100804426744942,
 0.3231423167429337,
 0.327202801584467,
 0.35100039119360915,
 0.3117060966356836,
 0.30915780310404717,
 0.3039476102853824,
 0.27520809495773724,
 0.3437910100579115,
 0.3161231351262217,
 0.3090703383784906,
 0.32747354302349746,
 0.29310817104548575,
 0.3233609314241303,
 0.33110932743505417,
 0.36316912376762606,
 0.30621974996788087,
 0.308370829493368,
 0.39303176336056767,
 0.33579762175784267,
 0.33619221424345963,
 0.31786159801243985,
 0.349243946616376,
 0.48671476566560945,
 0.29757383067661297,
 0.305565800853166,
 0.328459345880754,
 0.3467178307792631,
 0.28354652239824696,
 0.2896963639944273,
 0.3199756370924759,
 0.30014676503802074,
 0.31985660145212963,
 0.3035966935911462,
 0.34693278378371606,
 0.35080965122000385,
 0.3607900832164498,
 0.334645866040334,
 0.31479800995774054,
 0.3138631257530087,
 0.3319729427827464,
 0.37313740307497534,
 0.3265858459015677,
 0.3180330166016491,
 0.3322119521279865,
 0.3686306296880091,
 0.3589734779975828,
 0.28635035567062306,
 0.29349744369510433,
 0.31421292530363054,
 0.28042397644633277,
 0.35489414193462304,
 0.3143829324639156,
 0.34421127712653765,
 0.35962753492645266,
 0.30469797218174577,
 0.3096447700747837,
 0.3186648462479161,
 0.33934653110370955,
 0.5013702357364915,
 0.3188103126753092,
 0.3062093615526545,
 0.3298782525465638,
 0.31812745122475616,
 0.34013527499355245,
 0.31700790066492435,
 0.3348692705966547,
 0.32006456846010345,
 0.3053949173301128,
 0.3248024442101266,
 0.3270851917666629,
 0.4093172488817323,
 0.41406950941386333,
 0.4255280490693051,
 0.2918850722635804,
 0.3936605396829833,
 0.31347066225668846,
 0.2978803850058937,
 0.3304131760235399,
 0.30615841666801474,
 0.2941785087793266,
 0.4799375243523108,
 0.336974162246245,
 0.36042600551278126,
 0.3284078402301006,
 0.3324257109968481,
 0.3279738064987567,
 0.3269401570415175,
 0.3167882105893503,
 0.3370941246461897,
 0.3035910776259641,
 0.3256968568980069,
 0.32547972127905844,
 0.341342619493991,
 0.47950139507159883,
 0.344264691733226,
 0.3093870697105086,
 0.3547515897757721,
 0.4146927409818116,
 0.33446052387901637,
 0.74765233777123,
 0.31887570536074494,
 0.3413147774583958,
 0.3497031474318359,
 0.31583747488142067,
 0.3215035518460035,
 0.34800923125544087,
 0.33147932544203124,
 0.3223901624942542,
 0.3154628223543054,
 0.37324926583301804,
 0.3016102516797416,
 0.3030261795126884,
 0.32736800395363613,
 0.37859508712957113,
 0.3114560623958263,
 0.32464784247574185,
 0.3273219347747216,
 0.34728562651018724,
 0.3365441617532523,
 0.4341044076792636,
 0.3141458976719762,
 0.3928563367543652,
 0.3258503366550596,
 0.3215764020368198,
 0.31382014980550704,
 0.38078017141257603,
 0.3465582179141372,
 0.37323000597404776,
 0.32709390467632865,
 0.3242025814474186,
 0.45799101147123716,
 0.31342242285826233,
 0.3357664527549891,
 0.29363675701600955,
 0.4540897221715959,
 0.33860899816268897,
 0.33793242908359167,
 0.36706634150915113,
 0.3461006280623916,
 0.33829245393420443,
 0.29955114062453075,
 0.3086058711970913,
 0.3384866367877865,
 0.31583106029803965,
 0.3612938014505254,
 0.3046813124109362,
 0.3412383618151784,
 0.30686780228181354,
 0.4116871111612423,
 0.3192446080287428,
 0.3131393606415385,
 0.2894796676274358,
 0.30681708521065343,
 0.3190785749978633,
 0.3786528397174125,
 0.37015990311834057,
 0.33420103329133993,
 0.29873233195213783,
 0.3217277529155659,
 0.49078837733767816,
 0.3378181752018421,
 0.32206936140624415,
 0.32630817410490337,
 0.3290132493604216,
 0.3669673137889926,
 0.3400916287665972,
 0.29329711946179926,
 0.3412624207910067,
 0.3223826653343414,
 0.3488219364759158,
 0.3428072566055712,
 0.30812224839138413,
 0.3374636962224351,
 0.3408710198767388,
 0.4092379428544902,
 0.404742941944662,
 0.47130262111960664,
 0.3448569440539784,
 0.30175415473937484,
 0.3390823968522621,
 0.4012395287131327,
 0.3240333132046748,
 0.31281819899763014,
 0.32105784460224274,
 0.3277125399211405,
 0.37083353735794544,
 0.4340022402856394,
 0.3729185032775357,
 0.3300413696055681,
 0.2815665588053747,
 0.33372736774922546,
 0.3444584136971631,
 0.3110564573524868,
 0.35509789355930693,
 0.3261722853842492,
 0.34930425466485127,
 0.6505701042387082,
 0.3101502032982519,
 0.3337552783694906,
 0.5176128065250692,
 0.32586105582552405,
 0.5696075355850503,
 0.5117129281220164,
 0.37404899048986845,
 0.3571376866588431,
 0.31081638323457045,
 0.32921686491413166,
 0.3501945203219644,
 0.30573743089150357,
 0.29026726290950616,
 0.35511299460787693,
 0.332593576982413,
 0.4935484968250386,
 0.3471037690731399,
 0.3271164819259159,
 0.3015229563636025,
 0.392549228695617,
 0.35332353421457635,
 0.32724685085927013,
 0.3169039445268469,
 0.3230795262246483,
 0.4600863157533259,
 0.296814802922712,
 0.28523402867457465,
 0.33187081270133517,
 0.3767216847009091,
 0.29832580637025724,
 0.5567312613689126,
 0.3198826825298029,
 0.33700146330747993,
 0.3216700318750085,
 0.30663217533224574,
 0.32726260307880484,
 0.32596780913561646,
 0.4143969993994667,
 0.32594746093973626,
 0.5136442294066986,
 0.3974768531439781,
 0.44663845808635466,
 0.340769292292408,
 0.3207954363236709,
 0.33526313137811414,
 0.3134334672635268,
 0.3819029390763197,
 0.30707284761850806,
 0.3530865917355113,
 0.2789891281745459,
 0.30806944848797896,
 0.7484560491512791,
 0.3394486584947084,
 0.3051728393406027,
 0.34551218162982156,
 0.4456369503167718,
 0.33009433967298396,
 0.30356532678469395,
 0.31387267759286946,
 0.3344141410037184,
 0.3525544139999939,
 0.3103117915458757,
 0.36800290260466,
 0.3656768987318654,
 0.33467627358353375,
 0.3349889830408974,
 0.43401356324356194,
 0.30199621581398683,
 0.2799927251263896,
 0.33560322417280986,
 0.4102866378464596,
 0.36594149388971053,
 0.32623252922331164,
 0.37373699638934255,
 0.3732048846808514,
 0.36696797180412594,
 0.36923990792679834,
 0.33418991296879874,
 0.30752163402485827,
 0.3378442753452137,
 0.32977379679250235,
 0.3806455682821131,
 0.3014685338474806,
 0.3310193106830894,
 0.3146565625582833,
 0.32637714626910286,
 0.3692659048290468,
 0.37085221260950557,
 0.36525375760849177,
 0.325442027470267,
 0.32232090505950173,
 0.2927114188712367,
 0.3079252051759648,
 0.3467921309261432,
 0.34173376271330097,
 0.33905253805558094,
 0.3558275378932386,
 0.31438229149888947,
 0.33974707599389614,
 0.36635234394158894,
 0.45855658513809605,
 0.3399531355736823,
 0.29680629099153794,
 0.5055265738143648,
 0.5268544865366447,
 0.3912735736220314,
 0.34287160419067403,
 0.3306157642256253,
 0.31268901146944356,
 0.33120480445503386,
 0.46374467056067775,
 0.33283990931923546,
 0.3218074304238955,
 0.31795220725644013,
 0.35943988741451566,
 0.3299654834582118,
 0.33179609877986754,
 0.36158315960710463,
 0.3124739739346831,
 0.3008277962616927,
 0.291062449418331,
 0.33034070264529797,
 0.362680122157244,
 0.32865890659546554,
 0.2941849722283279,
 0.30668955400347997,
 0.29076425534099154,
 0.4400213168882412,
 0.33277664927638523,
 0.5030727796366211,
 0.32522447731663706,
 0.3337690261370101,
 0.28962135216200174,
 0.3237887082596901,
 0.29759149922141975,
 0.33767012759657333,
 0.33605769948876807,
 0.3086375286333018,
 0.3249983375560205,
 0.3120828141503976,
 0.3514658844351829,
 0.3192206247855716,
 0.3842627128173439,
 0.34401636036640837,
 0.5409646982695387,
 0.4526085608093215,
 0.3586122199583919,
 0.3361516485141719,
 0.3113424498403777,
 0.5140080243557859,
 0.2934258451954484,
 0.36934071911723115,
 0.2855012571542994,
 0.3048576018098731,
 0.3008563495960601,
 0.3417115637577563,
 0.3132861408959384,
 0.41618572342224985,
 0.28924873723472444,
 0.3050982374775504,
 0.4134467285077258,
 0.5236801714865946,
 0.32259241113695003,
 0.30370790226533023,
 0.2778241883424022,
 0.3431417633588014,
 0.31958457616786046,
 0.3262787866038658,
 0.3156308978257654,
 0.32901333832945456,
 0.3306716637146335,
 0.30414852923805547,
 0.34010220105900746,
 0.31382221466217086,
 0.3132062867409138,
 0.31853264506179907,
 0.3794527288527362,
 0.32892877557485145,
 0.3426399643968924,
 0.33619908979573054,
 0.3195161287761389,
 0.3498291648675016,
 0.6206762656329112,
 0.33725183420924565,
 0.29726051903386475,
 0.492266862036397,
 0.304422389886216,
 0.3240254628459157,
 0.3278321282256683,
 0.3214630854590693,
 0.2916831613003809,
 0.30592464145626913,
 0.32894904615579706,
 0.36056282716072113,
 0.3053484141356311,
 0.3601392727470132,
 0.34834129440576483,
 0.3102458802229211,
 0.3398631283952896,
 0.5069404607132209,
 0.3276700221943307,
 0.3264176271760335,
 0.3177934930717426,
 0.3100869498140981,
 0.3470304231541419,
 0.3163192896204212,
 0.29060943255975946,
 0.32467145267742786,
 0.3428483058725535,
 0.4490083708902179,
 0.31283792462515847,
 0.3065847898782012,
 0.32788106947776724,
 0.3652538107102409,
 0.37397593551754427,
 0.31695261831477745,
 0.3226826047952713,
 0.3810698915128567,
 0.3483956030593569,
 0.6298144270673348,
 0.3260259765123406,
 0.3098062832423649,
 0.30565821013233047,
 0.3684836255291011,
 0.3167628896942579,
 0.28004981331457823,
 0.3588594043602268,
 0.35006650094360203,
 0.47711870212644575,
 0.711126507532016,
 0.35364443496895476,
 0.2861877701805046,
 0.3597934044593806,
 0.43619630918529795,
 0.46216025251485504,
 0.33520277961913936,
 0.32493441214803037,
 0.3417863916575506,
 0.29088709651810235,
 0.3374584619608111,
 0.31955451160124354,
 0.27011746014656624,
 0.32739233395673734,
 0.3555217745980069,
 0.3217429990780488,
 0.38973347705493516,
 0.3378063402357498,
 0.5429610650923167,
 0.29740134576639926,
 0.3505602700480427,
 0.3687548335989331,
 0.3165013623635275,
 0.27140052437261886,
 0.3041480847158254,
 0.34562753698387244,
 0.3454007077338573,
 0.38037745395002015,
 0.31013515633640476,
 0.3300436630597105,
 0.3271999801020734,
 0.3048375323676027,
 0.3124132732674892,
 0.3326904019075608,
 0.31868294037055006,
 0.4707675569282038,
 0.2987020561109984,
 0.3454722273921587,
 0.31986601996676145,
 0.39830323854173943,
 0.36140344470374625,
 0.33178478494400904,
 0.31416219076664254,
 0.32683210926454564,
 0.3199035774480591,
 0.2774136274140324,
 0.3177229581636333,
 0.3512728001421562,
 0.3657653923331966,
 0.3094735631895282,
 0.3288004797350293,
 0.339617963552337,
 0.32562263112465367,
 0.47048523876499315,
 0.34483090745431527,
 0.4318515350895795,
 0.4471459528569286,
 0.3481886681279984,
 0.34370516550950647,
 0.3441491943384337,
 0.33527977674176673,
 0.3407551634255854,
 0.30423980801456424,
 0.30519691111375,
 0.29082775168075975,
 0.3781789736602268,
 0.3306088657571172,
 0.34011885737179604,
 0.33942078580882723,
 0.31058132779237413,
 0.3149608185895381,
 0.3310073837702955,
 0.32165137095114504,
 0.42437557992264663,
 0.3273331139550992,
 0.341201353233011,
 0.31890279220599776,
 0.31164755638161856,
 0.3098670284879755,
 0.3153365178876561,
 0.3287770638917408,
 0.7293400458649953,
 0.3316261065411185,
 0.6137425159767143,
 0.332071402273561,
 0.40502923862085716,
 0.3216585984342461,
 0.3156646621356808,
 0.5343214911392703,
 0.26370007983430677,
 1.1246047734055067,
 0.4239357800008362,
 0.43308135143225895,
 0.33161379306040306,
 0.30372638535283314,
 0.29867666221764255,
 0.36020670354477474,
 0.4945172317652339,
 0.4437759970966654,
 0.31177458551134063,
 0.30672090616744807,
 0.3728412853539673,
 0.42028985495248106,
 0.47463239180569883,
 0.3355176977237193,
 0.38934292638599904,
 0.32201451185953783,
 0.33473777453445114,
 0.598904763453687,
 0.3406810162492164,
 0.3445652996608646,
 0.30576189135993564,
 0.29682363479096424,
 0.3058476985113863,
 0.3237828950749089,
 0.3713105785851169,
 0.315938215989863,
 0.30951619819926157,
 0.3575203037758376,
 0.38160837847710455,
 0.6916990122199915,
 0.33527895131700836,
 0.3155739820177706,
 0.31923434573966486,
 0.4017223781056492,
 0.4715877581109094,
 0.3909039697916502,
 0.2841539531162263,
 0.3110268004232961,
 0.33600800688865917,
 0.37719731640803444,
 0.34400437563299724,
 0.4063140239837806,
 0.32123594742357736,
 0.3980952091498433,
 0.3203259555061472,
 0.38714119984499756,
 0.49545269603845693,
 0.4405944462489269,
 0.3600342136487449,
 0.35217918028172424,
 0.43776502931413896,
 0.46841362592147207,
 0.34701415423243265,
 0.31131280769616293,
 0.4461045160789253,
 0.28804256338614787,
 0.5051213283696289,
 0.3429703442019936,
 0.46178471494986756,
 0.4863291928049143,
 0.41270719538463546,
 0.39198067092793937,
 0.468428669122286,
 0.3300700971113327,
 0.36627899851377677,
 0.47178863797333875,
 0.4515381238164829,
 0.5457932551979093,
 0.6101091666559887,
 0.35708540577004116,
 0.649807831111971,
 0.445993507344448,
 0.33027080415719073,
 0.438582532083851,
 0.31406888827423113,
 0.3461106131148225,
 0.5049295847022957,
 0.5300916422899018,
 0.6183290798161454,
 0.5205686460991982,
 0.5588604177811206,
 0.5403206516376879,
 0.522323461780904,
 0.40554212656322125,
 1.4557743776675762,
 0.5470949835644173,
 0.4190710879374755,
 0.3489140484638139,
 0.4932467512994808,
 0.6374016994017146,
 0.35044526597341774,
 0.43603081288244266,
 0.5530436563598319,
 0.5433966815602701,
 1.5256874727733154,
 0.33773814032819166,
 0.3108680697099063,
 0.4895967406732917,
 0.5125543235727344,
 0.4798224100369252,
 0.5837160476391726,
 0.44287344694304354,
 0.47671473832935113,
 1.208600267162432,
 0.4266822445780353,
 0.47767279150894504,
 0.60913070646091,
 1.191793379361224]